In [12]:
import numpy as np
import sklearn.metrics as metrics
import os

from keras.datasets import cifar10
import keras.callbacks as callbacks
import keras.utils.np_utils as kutils
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import plot_model
import pydotplus as pydot
from keras import backend as K
import importlib
import wide_residual_network_RAM_v3 as wrn
import wide_residual_network_RAM_v3_freeze as wrn_freeze

importlib.reload(wrn)
importlib.reload(wrn1)
from keras.callbacks import History

In [15]:
#decathlon_data_folder = "C:\\Users\\Astha\\Dropbox\\data science\\nips paper implementation\\code\\data\\decathlon-1.0-data"
#tasks = os.listdir(decathlon_data_folder)
#nb_tasks = len(tasks)+1 #1 added for imagenet which is not included in this folder

In [16]:
batch_size = 8
nb_epoch = 3
img_rows, img_cols = 32,32
classes = 2

In [17]:
cwd = os.getcwd()
#data_folder = decathlon_data_folder + "\\aircraft\\"
data_folder = cwd + "\\sample_data\\catsdogs\\"

In [18]:
train_datagen = ImageDataGenerator(samplewise_center=True, samplewise_std_normalization=True)
valid_datagen = ImageDataGenerator(samplewise_center=True, samplewise_std_normalization=True)
train_generator = train_datagen.flow_from_directory(data_folder + "train", target_size=(img_rows, img_cols),batch_size=batch_size)
valid_generator = valid_datagen.flow_from_directory(data_folder + "val", target_size=(img_rows, img_cols),batch_size=batch_size)

Found 145 images belonging to 2 classes.
Found 15 images belonging to 2 classes.


In [19]:
init_shape = (3,img_rows, img_cols ) if K.image_dim_ordering() == 'th' else (img_rows, img_cols ,3)

In [22]:
# For WRN-16-8 put N = 2, k = 8
# For WRN-28-10 put N = 4, k = 10
# For WRN-40-4 put N = 6, k = 4
model = wrn_freeze.create_wide_residual_network(init_shape, nb_classes=classes, N=4, k=4, dropout=0.0, learnall = False)

model.summary()
plot_model(model, to_file = "WRN-28-4-RAM_v3_freeze.png")

Wide Residual Network-28-4 created.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_213 (Conv2D)             (None, 32, 32, 16)   432         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_213 (BatchN (None, 32, 32, 16)   64          conv2d_213[0][0]                 
__________________________________________________________________________________________________
conv2d_214 (Conv2D)             (None, 32, 32, 16)   256         batch_normalization_213[0][0]    
_________________________________________________________________________

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model).create(prog='dot', format='svg'))

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["acc"])
print("Finished compiling")
print("Allocating GPU memory")

#model.load_weights("weights/WRN-28-8 Weights.h5")
#print("Model loaded.")

In [ ]:
filepath = "sample.h5"
#model.fit_generator(generator.flow(trainX, trainY, batch_size=batch_size), steps_per_epoch=len(trainX) // batch_size + 1, nb_epoch=nb_epoch,callbacks = [callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)],validation_data=(testX, testY),validation_steps=testX.shape[0] // batch_size,)
from keras.callbacks import History

csv_logger = callbacks.CSVLogger('training.log', separator = ',', append = True)

model.fit_generator(train_generator, steps_per_epoch= len(train_generator), validation_data=valid_generator, validation_steps = len(valid_generator), epochs = nb_epoch, callbacks = [csv_logger, callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)])

In [ ]:
print(history.history.keys())

# Sample model for Keras testing

In [ ]:
from keras.models import Model
from keras.layers import Input, Add, Activation, Dropout, Flatten, Dense
from keras.layers.convolutional import Convolution2D, MaxPooling2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras import backend as K

def res_adapt_mod(input):
    init=input
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1
    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform')(input)
    x = Convolution2D(1, (3, 3), padding='same', kernel_initializer='he_normal',
                      use_bias=False)(x)
    x = Add()([x, init])

    return x  
def initial_conv(input):
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1
    x = Convolution2D(16, (3, 3), padding='same', kernel_initializer='he_normal',
                      use_bias=False, trainable = False)(input)
    x = res_adapt_mod(x)

    x = BatchNormalization(axis=channel_axis, momentum=0.1, epsilon=1e-5, gamma_initializer='uniform', trainable = False)(x)
    x = Activation('relu', trainable=False)(x)
    return x

In [ ]:
def create_wide_residual_network(input_dim, nb_classes=100, N=2, k=1, dropout=0.0, verbose=1):
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1

    ip = Input(shape=input_dim)

    x = initial_conv(ip)
    return x

In [ ]:
import wide_residual_network_RAM_frozen as wrn1
init_shape = (3,img_rows, img_cols ) if K.image_dim_ordering() == 'th' else (img_rows, img_cols ,3)

model = wrn1.create_wide_residual_network(init_shape, nb_classes=classes, N=4, k=4, dropout=0.0)
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["acc"])

In [ ]:
model.fit_generator(train_generator, steps_per_epoch= len(train_generator), validation_data=valid_generator, validation_steps = len(valid_generator), epochs = nb_epoch, callbacks = [ callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)])